# Deep Learning to solve a Client Success problem - Duty Manager call prediction deployment notebook

### Goal: predict tickets that will generate a Duty Manager (DM) call

- ~1.5% of Db2 tickets generate DM calls; each call generates additional work/impedence for the customer, the analyst, and the Duty Manager
- if we could predict which tickets will generate DM calls, and take proactive actions on those tickets (e.g. proactively call customer), all stakeholders benefit
- train a simple deep learning model on Db2 ticket data Feb 1 - May 20, 2018
- this notebook applies that model to tickets post May 20

### Features
- categorical: CASE_OWNER_ALIAS, SUPPORT_MISSION, CREATED_BY, ACCOUNT_NAME, LEGACY_PROBLEM_NUMBER, SEVERITY_LEVEL_NUMBER_FORMULA, OPERATING_SYSTEM, PRODUCT_NAME, ACCOUNT_PRIORITY, PRODUCT_VERSION, BLUE_DIAMOND_ACCOUNT
- continuous: OWNERSHIP_CHANGES
- text: SUBJECT

### Label / Target:

- will the ticket generate a DM call? 0 if no; 1 if yes

### Current HWMs: 87% accuracy on validation; 81.5% on "wild" data

### Supported Versions for deployment: 
- Keras version: 2.1.3
- TensorFlow version: 1.5

### Functionalities covered in this notebook:
- load a trained Keras model
- Save the trained Keras model to WML Repository
- Deploy and score using the saved model

### Pre-requisites to deploy a Keras model
- Keras's save() API should be used for saving the trained model.
- The saved model's .h5/.hdf5 file must be archived and compressed in tar.gz(/.tgz) format. The .h5/.hdf5 file must be at the first level in the compressed archive. 
- The compressed archive file must be saved in the WML Repository.


### Next steps to try:

- update model to incorporate DATE_OPENED feature
- deploy and maintain in production

DL Glossary: https://deeplearning4j.org/glossary


# Proposed end-to-end deployment of model
<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/machine-learning-may-2018/ml-project-edits/endtoend.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# Data transformations
<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/machine-learning-may-2018/ml-project-edits/datatransformationsjune11.jpg" width="600" alt="Icon"> </th>
   </tr>
</table>

# Visualization of model
<table style="border: none" align="left">
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://raw.githubusercontent.com/ryanmark1867/machine-learning-may-2018/master/dmmodelplotjune9annot.png" width="600" alt="Icon"> </th>
   </tr>
</table>

# Links to key parts of the notebook <a name='linkanchor' />
<a href=#ingestdash>Ingest data and load model</a>

<a href=#definecategories>Define feature categories</a>

<a href=#bookmark>Deal with missing values</a>

<a href=#modelfit>Define Keras variables</a>

<a href=#predrend>Predictions and renderings</a>

<a href=#confusionmatrix>Confusion matrix</a>

In [2]:
import zipfile
import pandas as pd
import numpy as np
import time
import datetime
from dateutil import relativedelta
from io import StringIO
import pandas as pd
from keras.models import model_from_json
from keras.models import load_model
from keras.models import model_from_yaml
import pickle
import tensorflow as tf
import keras
# from sklearn.preprocessing import CategoricalEncoder
# DSX code to import uploaded documents
from io import StringIO
import requests
import json
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.cross_validation import train_test_split
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt
%matplotlib inline 
import math
from subprocess import check_output
import seaborn as sns
import types
import tempfile
import keras.models
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Validate Keras level

In [3]:
!pip install keras==2.1.3

    100% |████████████████████████████████| 327kB 4.0MB/s eta 0:00:01
Requirement not upgraded as not directly required: scipy>=0.14 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from keras==2.1.3)
Requirement not upgraded as not directly required: numpy>=1.9.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from keras==2.1.3)
Requirement not upgraded as not directly required: six>=1.9.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from keras==2.1.3)
Requirement not upgraded as not directly required: pyyaml in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from keras==2.1.3)
  Found existing installation: Keras 2.1.4
    Uninstalling Keras-2.1.4:
      Successfully uninstalled Keras-2.1.4


In [4]:
!pip show keras tensorflow

Name: Keras
Version: 2.1.3
Summary: Deep Learning for humans
Home-page: https://github.com/keras-team/keras
Author: Francois Chollet
Author-email: francois.chollet@gmail.com
License: MIT
Location: /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requires: scipy, pyyaml, six, numpy
---
Name: tensorflow
Version: 1.3.0
Summary: TensorFlow helps the tensors flow
Home-page: http://tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requires: tensorflow-tensorboard, numpy, wheel, six, protobuf


In [5]:
def bootstrap():
    
    global dsxmode, csvmode, dbmode, pSpace, testproportion, trainproportion, verboseout, includetext, presaved, savemodel
    global remakeTokenizer, hctextmax, maxwords, BATCH_SIZE, savedle, targetthresh, emptythresh, class_weight
    global zero_weight, one_weight, hyperparameters, learning_rate, dropout_rate, l2_lambda
    dsxmode = False # overall is this being run in DSX?
    csvmode = True # ingest from CSV
    dbmode = False # ingest from database
    pSpace = False # pSpace mode

    testproportion = 0.01 # proportion of data reserved for test set
    trainproportion = 0.99 # proportion of non-test data dedicated to training (vs. validation)
    verboseout = True
    includetext = True # switch to determine whether text fields are included in model
    presaved = True # switch to determine whether to train model or load saved model
    savemodel = True # switch to determine whether to save model
    remakeTokenizer = False # switch to determine whether to rebuild tokenizer based on input in this notebook or use imported tokenizer
    hctextmax = 7000
    maxwords = 6000
    BATCH_SIZE = 200
    savedle = True # switch for whether a pickled label encoder is used

    targetthresh = 6.0
    emptythresh = 1000
    # to address imbalance in training data between zero (above targetthresh) and detractor (below targetthresh) specify weight in compile and fit
    # class_weight = {0 : zero_weight, 1: one_weight}
    # consider calculating these values from actual skew rather than hard-coding them here
    zero_weight = 1.0
    one_weight = 72.8

    # hyperparameters
    learning_rate = 0.001
    dropout_rate = 0.00003 #0.003
    l2_lambda = 0.00003 #7.5


In [6]:
# The code was removed by Watson Studio for sharing.

# Ingest data and load model<a name='ingestdash' />
<a href=#linkanchor>Back to link list</a>

In [7]:
# ingest data from duty manager report and all cases for the scoring period
def ingest_data():
    global dm_cases, merged_data
    dm_cases = pd.read_csv(project.get_file('dmcasesjun22018.csv'),encoding = "ISO-8859-1")
    merged_data = pd.read_csv(project.get_file('alldmeracasesjune22018posttrain.csv'),encoding = "ISO-8859-1")
    merged_data.columns = map(str.upper,merged_data.columns)
    dm_cases.columns = map(str.upper,dm_cases.columns)

In [8]:
# The code was removed by Watson Studio for sharing.

In [9]:
#loading model, weights, tokenizer and label encoder

def load_items():
    loaded_model = pickle.load(project.get_file('pickledmodelaug4.pickle'))
    #load_path = "dmmodelaug2.h5"
    #loaded_model = load_model(load_path)
    if remakeTokenizer == False:
        tok_raw = pickle.load(project.get_file('tokenizeraug4.pickle'))
        # workaround for issue serializing tokenizer
    tok_raw.oov_token = None
    lelist = pickle.load(project.get_file('leaug4.pickle'))
    nearempty = pickle.load(project.get_file('neaug4.pickle'))
    print("len(lelist)",len(lelist))
    return(loaded_model, tok_raw, lelist, nearempty)

In [10]:
# create merged dataframe
def create_merged_df(dm_cases, merged_data):
    # for all rows in the dm_cases dataframe set the target to 1
    dm_cases["target"] = 1
    # defined columns to keep from dm_cases
    dmcollist = list(dm_cases)
    dmexlist = ['CASE_NUMBER','target']
    dmcollist = list(set(dmcollist) - set(dmexlist))
    # drop extraneous columns from dm_cases
    dm_cases.drop(dmcollist, inplace=True, axis=1)
    print("dm_cases cols post pruning",list(dm_cases))
    print("dm_cases.target.value_counts()",dm_cases.target.value_counts())
    print("dm_cases.shape",dm_cases.shape)
    # join the dm_cases dataframe with the overall cases dataframe
    merged_data = merged_data.join(dm_cases.set_index('CASE_NUMBER'), on = 'CASE_NUMBER')
    print("merged_data.shape",merged_data.shape)
    print("merged_data.target.value_counts()",merged_data.target.value_counts())
    # fill target column with 0 if not already 1
    merged_data['target']=merged_data['target'].fillna(0.0)
    print("merged_data.target.value_counts()",merged_data.target.value_counts())
    print("merged_data.shape",merged_data.shape)
    # clean up LEGACY_PROBLEM_NUMBER
    merged_data['LEGACY_PROBLEM_NUMBER'] = np.where(merged_data['LEGACY_PROBLEM_NUMBER'] == 'Not Applicable','Not Applicable','PMR')
    if presaved == True:
    # if the model is saved all data gets processed
        train = merged_data
        print("presaved model - not training/test split")
    else:
        train, test = train_test_split(merged_data, test_size = testproportion)
        print(testproportion)
        print("Through train test split. Test proportion:")
    return(merged_data, train)

# Define feature categories <a name='definecategories' />
<a href=#linkanchor>Back to link list</a>

In [11]:
# define the column lists for categorical, continuous, and text features, as well as columns to exclude from running through the model

def define_feature_categories(merged_data):
    allcols = list(merged_data)
    print("all cols",allcols)
    textcols = ['SUBJECT'] # columns to deal with as text - replace entries with multiple IDs and use embeddings, RNN
    continuouscols = ['OWNERSHIP_CHANGES'] # columns to deal with as continuous values - no embeddings
    excludefromcolist = ['CASE_NUMBER','target','OPENED_DATE'] # columns to exclude completely from the model
    # list(set(temp1) - set(temp2))
    nontextcols = list(set(allcols) - set(textcols))
    collist = list(set(nontextcols) - set(excludefromcolist) - set(nearempty) - set(continuouscols))
    # ensure continuous categories have numeric type
    for col in continuouscols:
        merged_data[col] = merged_data[col].astype(float)

    # print column list contents:
    print("allcols",allcols)
    print("nearempty",nearempty)
    print("excludefromcolist",excludefromcolist)
    print("textcols",textcols)
    print("continuouscols",continuouscols)
    # Aug 17 hardcode collist for testing
    #collist = ['OPERATING_SYSTEM', 'ACCOUNT_PRIORITY', 'BLUE_DIAMOND_ACCOUNT', 'SEVERITY_LEVEL_NUMBER_FORMULA', 'PRODUCT_VERSION', 'PRODUCT_NAME', 'SUPPORT_MISSION', 'CREATED_BY', 'ACCOUNT_NAME', 'LEGACY_PROBLEM_NUMBER', 'CASE_OWNER_ALIAS']
    # Aug 28 hardcode collist to match order of input layers in model summary below
    # collist = ['ACCOUNT_NAME','CASE_OWNER_ALIAS','SEVERITY_LEVEL_NUMBER_FORMULA','PRODUCT_NAME','BLUE_DIAMOND_ACCOUNT','LEGACY_PROBLEM_NUMBER','PRODUCT_VERSION','CREATED_BY','SUPPORT_MISSION','ACCOUNT_PRIORITY','OPERATING_SYSTEM']
    # Aug 31 experiment - flip 2nd and 8th cols
    #collist = ['ACCOUNT_NAME','CREATED_BY','SEVERITY_LEVEL_NUMBER_FORMULA','PRODUCT_NAME','BLUE_DIAMOND_ACCOUNT','LEGACY_PROBLEM_NUMBER','PRODUCT_VERSION','CASE_OWNER_ALIAS','SUPPORT_MISSION','ACCOUNT_PRIORITY','OPERATING_SYSTEM']
    
    # Aug 31 - copy exact order of collist from model build
    # collist = ['OPERATING_SYSTEM','ACCOUNT_PRIORITY','BLUE_DIAMOND_ACCOUNT','SEVERITY_LEVEL_NUMBER_FORMULA','PRODUCT_VERSION','PRODUCT_NAME','SUPPORT_MISSION','CREATED_BY','ACCOUNT_NAME','LEGACY_PROBLEM_NUMBER','CASE_OWNER_ALIAS']
    # SEPT 2 - TRANSPOSE TWO PROBLEM COLS - PRODUCT_VERSION into LEGACY_PROBLEM_NUMBER
    # collist = ['OPERATING_SYSTEM','ACCOUNT_PRIORITY','BLUE_DIAMOND_ACCOUNT','SEVERITY_LEVEL_NUMBER_FORMULA','LEGACY_PROBLEM_NUMBER','PRODUCT_NAME','SUPPORT_MISSION','CREATED_BY','ACCOUNT_NAME','PRODUCT_VERSION','CASE_OWNER_ALIAS']
    #collist = ['OPERATING_SYSTEM','ACCOUNT_PRIORITY','BLUE_DIAMOND_ACCOUNT','SEVERITY_LEVEL_NUMBER_FORMULA','PRODUCT_VERSION','PRODUCT_NAME','SUPPORT_MISSION','CREATED_BY','ACCOUNT_NAME','LEGACY_PROBLEM_NUMBER','CASE_OWNER_ALIAS']
    # exact copy of order of columns from summary of model def notebook
    collist = ['ACCOUNT_NAME','CASE_OWNER_ALIAS','SEVERITY_LEVEL_NUMBER_FORMULA','PRODUCT_NAME','BLUE_DIAMOND_ACCOUNT','LEGACY_PROBLEM_NUMBER','PRODUCT_VERSION','CREATED_BY','SUPPORT_MISSION','ACCOUNT_PRIORITY','OPERATING_SYSTEM']
    
    # Sept 3 try reversing order - nope
    # collist = ['OPERATING_SYSTEM','ACCOUNT_PRIORITY','SUPPORT_MISSION','CREATED_BY','PRODUCT_VERSION','LEGACY_PROBLEM_NUMBER','BLUE_DIAMOND_ACCOUNT','PRODUCT_NAME','SEVERITY_LEVEL_NUMBER_FORMULA','CASE_OWNER_ALIAS','ACCOUNT_NAME']
    # experiment putting BD first
    # collist = ['BLUE_DIAMOND_ACCOUNT','ACCOUNT_NAME','CASE_OWNER_ALIAS','SEVERITY_LEVEL_NUMBER_FORMULA','PRODUCT_NAME','LEGACY_PROBLEM_NUMBER','PRODUCT_VERSION','CREATED_BY','SUPPORT_MISSION','ACCOUNT_PRIORITY','OPERATING_SYSTEM']
    # collist = ['BLUE_DIAMOND_ACCOUNT','ACCOUNT_NAME','CASE_OWNER_ALIAS','SEVERITY_LEVEL_NUMBER_FORMULA','PRODUCT_NAME','LEGACY_PROBLEM_NUMBER','PRODUCT_VERSION','CREATED_BY','SUPPORT_MISSION','ACCOUNT_PRIORITY','OPERATING_SYSTEM']
    # aug 31 experiment - blows up
    # collist = ['BLUE_DIAMOND_ACCOUNT','BLUE_DIAMOND_ACCOUNT','BLUE_DIAMOND_ACCOUNT','BLUE_DIAMOND_ACCOUNT','BLUE_DIAMOND_ACCOUNT','BLUE_DIAMOND_ACCOUNT','BLUE_DIAMOND_ACCOUNT','BLUE_DIAMOND_ACCOUNT','BLUE_DIAMOND_ACCOUNT','BLUE_DIAMOND_ACCOUNT','BLUE_DIAMOND_ACCOUNT']
    print("collist",collist)
    return(allcols,textcols,continuouscols,collist)

# Deal with missing values and encode categorical values and text <a name='bookmark' />
<a href=#linkanchor>Back to link list</a>

In [12]:
# fill missing values according to column type

def fill_missing(dataset):
    print("before mv")
    for col in collist:
        dataset[col].fillna(value="missing", inplace=True)
    for col in continuouscols:
        dataset[col].fillna(value=0.0,inplace=True)
    for col in textcols:
        dataset[col].fillna(value="missing", inplace=True)
    return (dataset)


In [13]:
# process categorical data
# replace values with category IDs in the following columns
# with the label encoder being read from LE used in training, do not fit again

def encode_labels(train,lelist):
    le = LabelEncoder()
    train['SUBJECT'].head(15)
    if savedle == True:
        # use lelist which was read from pickled version saved at model building time
        for col in collist:
            print("processing ",col)
            # le.fit(np.hstack([train[col], test[col]]))
            train[col] = lelist[col].transform(train[col])
    else:
        # create le based on test input                                      
        for col in collist:
            if verboseout:
                print("processing ",col)
            le.fit(np.hstack([train[col], test[col]]))
            train[col] = le.transform(train[col])
    train.iloc[:,0:13].head()
    return(train)

In [14]:
# tokenize list of text columns (made up of multiple strings)

from keras.preprocessing.text import Tokenizer

def encode_text(train, tok_raw):
    if remakeTokenizer == True:
        for col in textcols:
            if verboseout:
                print("processing text col",col)
            # Tokenizer lower cases and removes punctuation by default
            tok_raw = Tokenizer(num_words=maxwords,lower=True)
            tok_raw.fit_on_texts(train[col])
            train[col] = tok_raw.texts_to_sequences(train[col])
    else:
        for col in textcols:
            if verboseout:
                print("processing text col",col)
            # Tokenizer lower cases and removes punctuation by default
            print("using loaded tokenizer")
            train[col] = tok_raw.texts_to_sequences(train[col])
    train.iloc[:,0:13].head()
    return(train)

In [15]:
# calculate max values for embeddings


def get_max_embeddings(train):
    max_dict = {}
    textmax = 50
    i = 0
    for col in collist:
        
        max_dict[col] = np.max([train[col].max(), train[col].max()])+ 1
        # max_dict[col] = np.max([train[col].max(), train[col].max()])+2
        print("col in max is", col)
        print("max is", max_dict[col])
        print("i is ",i)
        i = i+1

    # np.max([np.max(train['Resolution_Description'].max()), np.max(train['Subject'].max()),np.max(train['Other_Reason_for_Cancellation'].max()),np.max(train['Reason_for_Reopening'].max())])) 
    for cols in textcols:
        # updated Aug 5 to clean up size mismatch
        max_dict[cols] = maxwords - 1 
        print("col is", cols)
        print("max is", max_dict[cols])
        '''
        maxtrain = np.max(train[(train[cols].map(len) != 0)][cols].map(max))
        #maxtest = np.max(test[(test[cols].map(len) != 0)][cols].map(max))
        if verboseout:
            print("maxtrain.max()",maxtrain)
            #print("maxtest .max()",maxtest)
        max_dict[cols] = maxtrain
        if max_dict[cols] > textmax:
            textmax = max_dict[cols]
        '''

    if textmax < hctextmax:
        textmax = hctextmax
    print("textmax",textmax)
    if verboseout:
        print("max_dict",max_dict)
    # maxwords
    return(max_dict, textmax)
    # return(maxwords, textmax)

# Define Keras variables <a name='modelfit' />
<a href=#linkanchor>Back to link list</a>

In [16]:
# define keras variables

# define keras variables
from keras.preprocessing.sequence import pad_sequences

# X for the features used

def get_keras_vars(dataset):
    X = {}
    dictlist = []
    i = 0
    for col in collist:
        if verboseout:
            print("cat col is",col)
            
        X[col] = np.array(dataset[col])
        dictlist.append(np.array(dataset[col]))
       
    for col in textcols:
        if verboseout:
            print("text col is",col)
        X[col] = pad_sequences(dataset[col], maxlen=max_dict[col])
        dictlist.append(pad_sequences(dataset[col], maxlen=max_dict[col]))
        
    for col in continuouscols:
        if verboseout:
            print("cont col is",col)
        X[col] = np.array(dataset[col])
        dictlist.append(np.array(dataset[col]))
        
    return X, dictlist

def get_keras_list_only(X_in):
    dictlist = []
    for key, value in X_in.items():
        print("X def loop key",key)
        print("value shape",value.shape)
        temp = [key,value]
        dictlist.append(value)
    return dictlist

def get_keras_np(X_in):
    return np.array(list(X_in.items()),dtype=object)
# np.array(list(result.items()), dtype=dtype)

# the deployment API for Watson Studio can only take a list/array, not a dictionary, so define list-only version for input




# Predictions and renderings <a name='predrend' />
<a href=#linkanchor>Back to link list</a>

In [17]:
# main cell to invoke functions

# get project token
def_project()
# initialize switches
bootstrap()
# get data
ingest_data()
# prep for unpickling of Keras model
make_keras_picklable()
# load model, tokenizer, label encodings, and exclusion list
loaded_model, tok_raw, lelist, nearempty = load_items()
# build combined dataframe
merged_data, train = create_merged_df(dm_cases, merged_data)
merged_data.iloc[:,0:13].head()
# define feature categories and ensure continuous cols have appropriate type
allcols,textcols,continuouscols,collist = define_feature_categories(merged_data)
train = fill_missing(train)
train = encode_labels(train, lelist)
train = encode_text(train, tok_raw)
# get max values for embeddings
# Sept 3 - don't define maxes - breaks creation of Keras variables
max_dict,textmax = get_max_embeddings(train)
dtrain = train
# print counts of target values for data sets
print("zero target values:",(dtrain["target"]==0).sum())
print("one target values:",(dtrain["target"]==1).sum())
# X_train = get_keras_vars(dtrain)
X_trainb, X_train_list = get_keras_vars(dtrain)

# preds = loaded_model.predict(X_train_list, batch_size=BATCH_SIZE)

len(lelist) 11
dm_cases cols post pruning ['CASE_NUMBER', 'target']
dm_cases.target.value_counts() 1    121
Name: target, dtype: int64
dm_cases.shape (121, 2)
merged_data.shape (1041, 16)
merged_data.target.value_counts() 1.0    11
Name: target, dtype: int64
merged_data.target.value_counts() 0.0    1030
1.0      11
Name: target, dtype: int64
merged_data.shape (1041, 16)
presaved model - not training/test split
all cols ['CASE_NUMBER', 'CASE_OWNER_ALIAS', 'OPENED_DATE', 'SUPPORT_MISSION', 'PRODUCT_NAME', 'ACCOUNT_PRIORITY', 'OWNERSHIP_CHANGES', 'SEVERITY_LEVEL_NUMBER_FORMULA', 'ACCOUNT_NAME', 'BLUE_DIAMOND_ACCOUNT', 'PRODUCT_VERSION', 'OPERATING_SYSTEM', 'SUBJECT', 'CREATED_BY', 'LEGACY_PROBLEM_NUMBER', 'target']
allcols ['CASE_NUMBER', 'CASE_OWNER_ALIAS', 'OPENED_DATE', 'SUPPORT_MISSION', 'PRODUCT_NAME', 'ACCOUNT_PRIORITY', 'OWNERSHIP_CHANGES', 'SEVERITY_LEVEL_NUMBER_FORMULA', 'ACCOUNT_NAME', 'BLUE_DIAMOND_ACCOUNT', 'PRODUCT_VERSION', 'OPERATING_SYSTEM', 'SUBJECT', 'CREATED_BY', 'LEGAC

In [18]:
loaded_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ACCOUNT_NAME (InputLayer)       (None, 1)            0                                            
__________________________________________________________________________________________________
CASE_OWNER_ALIAS (InputLayer)   (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_16 (Embedding)        (None, 1, 10)        22150       ACCOUNT_NAME[0][0]               
__________________________________________________________________________________________________
embedding_20 (Embedding)        (None, 1, 10)        2690        CASE_OWNER_ALIAS[0][0]           
__________________________________________________________________________________________________
SEVERITY_L

In [21]:
preds = loaded_model.predict(X_train_list, batch_size=BATCH_SIZE)

InvalidArgumentError: indices[0,0] = 438 is not in [0, 269)
	 [[Node: embedding_20/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](embedding_20/embeddings/read, embedding_20/Cast)]]

Caused by op 'embedding_20/Gather', defined at:
  File "/opt/conda/envs/DSX-Python35/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/envs/DSX-Python35/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/opt/conda/envs/DSX-Python35/lib/p

In [ ]:
print(X_train_list)

In [ ]:
print(X_train_list)

In [ ]:
print(X_train_list)

In [ ]:
preds.shape

In [ ]:
len(X_train_list)

In [ ]:
for i in X_train_list:
    print(i.shape)

In [ ]:
dtrain["predict"] = preds
dtrain.predict[:5]
if verboseout:
    dtrain.predict.hist()

In [ ]:
# get rounded predictions 
dtrain["predround"] = preds.round().astype(int)
dtrain.predround[:5]

In [ ]:
# get delta between predictions on training set and actual training target values
# hand calculate accuracy on training set as ratio of (total training samples - wrong training predictions)/total training samples

deltatr = abs(dtrain.target[:100000] - dtrain.predround[:100000])
deltatr[:50]
print(deltatr.sum())
print("percentage correct train")
print((len(deltatr) - deltatr.sum())/len(deltatr))

# Confusion matrix <a name='confusionmatrix' />
<a href=#linkanchor>Back to link list</a>

In [ ]:
from sklearn import metrics

cfmap=metrics.confusion_matrix(y_true=dtrain['target'],  # True labels
                         y_pred=dtrain["predround"])

label = ["0", "1"]
sns.heatmap(cfmap, annot = True, xticklabels = label, yticklabels = label)
plt.xlabel("Prediction")
plt.title("Confusion Matrix for DM prediction (weighted)")
plt.show()

In [ ]:
type(X_train)

In [ ]:
X_train

# Deploy model

In [ ]:
import keras
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import mnist
from keras.models import Sequential, load_model

from keras import backend as K
import numpy as np

'''
batch_size = 128
num_classes = 10
epochs = 1

# input shape
img_rows, img_cols = 28, 28

# samples to train
num_train_samples = 500

print(K._backend)

# prepare train and test datasets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# normalize the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train = x_train[:num_train_samples]
y_train = y_train[:num_train_samples]
print(x_train.shape)
print(y_train.shape)


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Model Definition
X = Input(shape=(28, 28, 1))
l1 = Conv2D(32, kernel_size=(3, 3),
            activation='relu')(X)
l2 = Conv2D(64, (3, 3), activation='relu')(l1)
l21 = MaxPooling2D(pool_size=(2, 2))(l2)
l22 = Dropout(0.25)(l21)
l23 = Flatten()(l22)

l3 = Dense(128, activation='relu')(l23)
l31 = Dropout(0.5)(l3)
y_hat = Dense(num_classes, activation='softmax', name="y_hat")(l31)

model = Model(inputs=X, outputs=y_hat)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


# Train a model
model.fit(x=x_train,
          y=y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))
print("model trained : " + str(type(model)))
# prediction_prob = model.predict(x_score)
# print(prediction_prob)
'''
# Save model
save_path = "dm_predict_modelj3c.h5"
loaded_model.save(save_path)



#### Create compressed archive of the saved model

In [ ]:
ls -ltr dm_*

In [ ]:
!tar -zcvf dm_predict_modelj3c.h5.tgz dm_predict_modelj3c.h5

In [ ]:
ls -ltr dm_*

## 2.0 Save the trained model to WML Repository

# Use `watson_machine_learning_client` Python library to save the trained model to WML Repository, to deploy the saved model and to make predictions using the deployed model.</br>


`watson_machine_learning_client` can be installed using the following `pip` command:

`!pip install watson-machine-learning-client --upgrade`

In [ ]:
!pip show watson-machine-learning-client 

In [ ]:
mkdir temp_install

In [ ]:
#!pip install watson-machine-learning-client --upgrade
!pip install -b ./temp_install --upgrade --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.python.org/simple/ watson-machine-learning-client==1.0.133
        

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
import json

In [ ]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "IBM", 
               # client.repository.ModelMetaNames.AUTHOR_EMAIL: "ibm@ibm.com", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "mryan@ca.ibm.com", 
               client.repository.ModelMetaNames.NAME: "DM_predict_kerasj3c",
               client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
               client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.5" ,
               client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [{"name": "keras", "version": "2.1.3"}]
              }

In [ ]:
!ls -ltr | grep dm

In [ ]:
# for model parameter, specify the .tgz file that you created with tar command
published_model = client.repository.store_model(model="dm_predict_modelj3c.h5.tgz", meta_props=model_props)


In [ ]:
import json

In [ ]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

In [ ]:
### Delete the model from WML Repository
### client.repository.delete(published_model_uid) 

## 3.0 Deploy the Keras model

In [ ]:
client.deployments.list()

In [ ]:
# client.deployments.delete("0ddbe9e4-f538-46e3-abce-59c3d3620cc6")

In [ ]:
created_deployment = client.deployments.create(published_model_uid, name="dm_predict_kerasj3c")
# KK3_clt_keras_mnist_mark

## 4.0 Predict using the deployed model

In [ ]:
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)

print(scoring_endpoint)

In [ ]:
# X_train is a dictionary - for each element the value is a large array of values
# X_train = get_keras_vars(dtrain)

'''
 for col in collist:
        X[col] = np.array(dataset[col])
        if verboseout:
            print("cat col is",col)
            print("shape is",X[col].shape)
  
  Result is:
            
cat col is LEGACY_PROBLEM_NUMBER
shape is (1041,)
cat col is ACCOUNT_NAME
shape is (1041,)
cat col is CREATED_BY
shape is (1041,)
cat col is ACCOUNT_PRIORITY
shape is (1041,)
cat col is SEVERITY_LEVEL_NUMBER_FORMULA
shape is (1041,)
cat col is PRODUCT_VERSION
shape is (1041,)
cat col is PRODUCT_NAME
shape is (1041,)
cat col is CASE_OWNER_ALIAS
shape is (1041,)
cat col is SUPPORT_MISSION
shape is (1041,)
cat col is BLUE_DIAMOND_ACCOUNT
shape is (1041,)
cat col is OPERATING_SYSTEM
shape is (1041,)
cont col is OWNERSHIP_CHANGES
shape is (1041,)
text col is SUBJECT
shape is (1041, 5999)'''

In [ ]:
X_train["LEGACY_PROBLEM_NUMBER"][4]

In [ ]:
'''
(x_train, y_train), (x_test, y_test) = mnist.load_data()
mnist.load_data() returns Tuple of Numpy arrays: (x_train, y_train), (x_test, y_test)
type(x_test): numpy.ndarray
x_test.shape: (10000, 28, 28, 1)
x_test[23].shape: (28, 28, 1)
type(x_test[23]): numpy.ndarray
type(x_score_1): list

and from score documentation:
>>> scoring_payload = {'fields': ['GENDER','AGE','MARITAL_STATUS','PROFESSION'], 'values': [['M',23,'Single','Student'],['M',55,'Single','Executive']]}
>>> predictions = client.deployments.score(scoring_url, scoring_payload)


x_score_1 = x_test[23].tolist()
x_score_2 = x_test[32].tolist()
'''
# from own code this is how model gets invoked
# preds = loaded_model.predict(X_train, batch_size=BATCH_SIZE)
# x_score_1 = X_train[23].tolist()
# x_score_2 = X_train[32].tolist()
# scoring_payload = {'values': [x_score_1, x_score_2]}
# try to create a list for items for
i = 0
lister = []
for col in collist:
    # L.append(obj)
    # X_train["LEGACY_PROBLEM_NUMBER"][4]
    lister.append(X_train[col][i])
        
for col in continuouscols:
    lister.append(X_train[col][i])
            
    
for col in textcols:
    lister.append(X_train[col][i])


scoring_payload = {'values': lister}

In [ ]:
lister

In [ ]:
predictions = client.deployments.score(scoring_endpoint, scoring_payload)


In [ ]:
print(json.dumps(predictions, indent=2))

### Delete Deployment
Please ensure to delete the Keras deployments if they are no longer required. 

In [ ]:
client.deployments.delete(client.deployments.get_uid(created_deployment))

# Kaggle submission that was used as input for this notebook
https://www.kaggle.com/knowledgegrappler/a-simple-nn-solution-with-keras-0-48611-pl

# Summary
This notebook shows loading and deployment of a Keras model for predicting Duty Manager calls.

# Author

Mark Ryan is a manager at IBM Canada.

Copyright © IBM Corp. 2018. This notebook and its source code are released under the terms of the MIT License.